In [85]:
import psycopg2
import pandas as pd
import numpy as np
import glob
import random
import glob
import os
import openpyxl
import re
from fuzzywuzzy import fuzz
from collections import defaultdict

# Project Workflow: User Performance Analysis

### Data Enginerring
    1. Some data will be in the spreadhseets because they were not all updated into my postgres instance. The remaining will be in my postgres instance.
        * Extract spreadsheet data with existing functions and calls and use psycopg2 to extract data from postgres instance. Combine into a single Pandas dataframe.

### Data Analysis
    1. Create some individual user dashboards and one large dashboard
    2. Visualize elements like volume load over time, most frequent exercises prescribed, most frequent exercises skipped, how many workouts (from postgres) were completed, how many workouts were skipped. 
    3. Comparison between loaded and bodywieght exercises

In [30]:
#Extracting prescription data from database

conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

#For business analysis
cursor.execute("""SELECT c.name AS NAME, 
workout_number, block_id, b.workouts_per_week, 
e.exercise AS exercise_name, sets, reps, p.weight 
FROM prescriptions p 
LEFT JOIN client c on p.client_id=c.id
LEFT JOIN exercises e on p.exercise_id=e.id
LEFT JOIN blocks b on p.block_id=b.id
WHERE c.name not in  ('Test','Jared')
ORDER BY c.name, block_id, workout_number;
""")
db_prescriptions=cursor.fetchall()
conn.close()

In [31]:
db_prescriptions=pd.DataFrame(db_prescriptions, columns=['name','workout_number','block_id','workouts_per_week','exercise_name','sets','reps','weight'])

In [32]:
db_prescriptions

,name,workout_number,block_id,workouts_per_week,exercise_name,sets,reps,weight
0,Abelev,0,14,2,Mountain Climbers,3,30,1
1,Abelev,0,14,2,Banded Walks,3,60,1
2,Abelev,0,14,2,Split Squat Jumps,3,10,1
3,Abelev,0,14,2,Banded Squats,3,10,1
4,Abelev,0,14,2,RFE Lunges,3,10,1
...,...,...,...,...,...,...,...,...
2715,Yelena,15,27,4,Mountain Climbers,3,30,1
2716,Yelena,15,27,4,Back Squats,4,5,50
2717,Yelena,15,27,4,Russian Twists,3,30,1
2718,Yelena,15,27,4,RFE Lunges,3,8,1


In [33]:
%store -r all_formatted_data
%store -r names

all_formatted_data=all_formatted_data
names=names

In [34]:
char_to_remove = [str(i) for i in range(10)]
specials_to_remove = ['*', '/', '**', '***', '(', '[', ')', ']', '&']
yelenas_to_remove = ['did', 'doing', 'didnt', 'need', 'could', 'couldnt', '?']
emphasis_to_remove = ['emphasis', 'Emphasis', 'Focus', 'Condition', 'recon', 'recondition', 'Recondition', 'wedding']

def passes_filters(exercise):
    if any(char in exercise for char in char_to_remove):
        return False
    if any(char in exercise for char in specials_to_remove):
        return False
    if any(char in exercise for char in yelenas_to_remove):
        return False
    if any(char in exercise for char in emphasis_to_remove):
        return False
    return True

In [35]:
def convert_to_int(data):
    try:
        for item in data:
            item['weight'] = int(float(item['weight']))
            item['sets'] = int(float(item['sets']))
            item['reps'] = int(float(item['reps']))
    except ValueError:
        return None

    return data
            

In [36]:
cleaned_dict={}
for client in all_formatted_data:
    client_data=[]
    for workout in all_formatted_data[client]:
        for rows in all_formatted_data[client][workout]:
            client_data.append(rows)
    cleaned_dict[client]=client_data

In [37]:
filtered_dict={}
for client in cleaned_dict:
    flitered_client=[]
    try:
        for item in cleaned_dict[client]:
            if passes_filters(item['exercise']):
                flitered_client.append(item)
    except:
        pass
    filtered_dict[client]=flitered_client

In [38]:
for client in filtered_dict:
    converted_data = convert_to_int(filtered_dict[client])
    filtered_dict[client] = converted_data

In [39]:
filtered_dict.keys()

dict_keys(['Dimitri', 'Masha', 'Misha', 'Anya', 'Arne', 'Max', 'Dino', 'Test', 'Ilya', 'Igor', 'Nick_Mary', 'Daniel Murphy', 'Yelena', 'Shawn'])

In [40]:
filtered_dict.pop('Test', None)

[]

In [41]:
old_key = 'Daniel Murphy'
new_key = 'Daniel'

if old_key in filtered_dict:
    filtered_dict[new_key] = filtered_dict[old_key]
    del filtered_dict[old_key]

In [42]:
old_key = 'Max'
new_key = 'Abelev'

if old_key in filtered_dict:
    filtered_dict[new_key] = filtered_dict[old_key]
    del filtered_dict[old_key]

In [43]:
filtered_dict['Abelev']

[{'exercise': 'J-Jacks', 'weight': 1, 'sets': 1, 'reps': 50},
 {'exercise': 'Foam Roll', 'weight': 1, 'sets': 1, 'reps': 1},
 {'exercise': 'Lb Warm Up', 'weight': 1, 'sets': 2, 'reps': 1},
 {'exercise': 'Static Lunges', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'Supermans', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'Shoulder Taps', 'weight': 1, 'sets': 3, 'reps': 30},
 {'exercise': 'Burpees', 'weight': 1, 'sets': 1, 'reps': 10},
 {'exercise': 'Stretches', 'weight': 1, 'sets': 1, 'reps': 1},
 {'exercise': 'J-Jacks', 'weight': 1, 'sets': 1, 'reps': 50},
 {'exercise': 'Foam Roll', 'weight': 1, 'sets': 1, 'reps': 1},
 {'exercise': 'Lb Warm Up', 'weight': 1, 'sets': 2, 'reps': 1},
 {'exercise': 'Static Lunges', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'Supermans', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'Burpees', 'weight': 1, 'sets': 2, 'reps': 10},
 {'exercise': 'Stretches', 'weight': 1, 'sets': 1, 'reps': 1},
 {'exercise': 'J-Jacks', 'weight': 1, 's

In [44]:
dfs=[]
for client in filtered_dict:
    df=pd.DataFrame(filtered_dict[client])
    dfs.append(df)

df=pd.concat(dfs, axis=0)
    

In [45]:
df

,exercise,weight,sets,reps
0,Repeat Squat Jumps,1,3,10
1,Split Squat Jumps,1,2,10
2,Lateral Walks,1,3,1
3,Banded Goblet Squats,1,3,10
4,Goblet Lunges,1,3,10
...,...,...,...,...
499,Repeat Squat Jumps,1,3,10
500,Split Squat Jumps,1,3,10
501,Dead Bugs,1,3,10
502,Plank Circuit,1,3,10


In [47]:
db_prescriptions=db_prescriptions.drop(columns=['name', 'workout_number','block_id','workouts_per_week'])
db_prescriptions

In [51]:
#Reordering so can concatenatye with df
db_prescriptions=db_prescriptions.rename(columns={'exercise_name':'exercise'})
db_prescriptions=db_prescriptions[['exercise','weight','sets','reps']]


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [55]:
db_prescriptions=db_prescriptions.reset_index(drop=True)
df=df.reset_index(drop=True)


,0,1,2,3,4,5,6,7
0,Repeat Squat Jumps,1,3,10,Mountain Climbers,1.0,3.0,30.0
1,Split Squat Jumps,1,2,10,Banded Walks,1.0,3.0,60.0
2,Lateral Walks,1,3,1,Split Squat Jumps,1.0,3.0,10.0
3,Banded Goblet Squats,1,3,10,Banded Squats,1.0,3.0,10.0
4,Goblet Lunges,1,3,10,RFE Lunges,1.0,3.0,10.0
...,...,...,...,...,...,...,...,...
6009,Repeat Squat Jumps,1,3,10,NaN,NaN,NaN,NaN
6010,Split Squat Jumps,1,3,10,NaN,NaN,NaN,NaN
6011,Dead Bugs,1,3,10,NaN,NaN,NaN,NaN
6012,Plank Circuit,1,3,10,NaN,NaN,NaN,NaN


In [56]:
big_df=pd.concat([df, db_prescriptions], axis=0, ignore_index=True)
big_df

,exercise,weight,sets,reps
0,Repeat Squat Jumps,1,3,10
1,Split Squat Jumps,1,2,10
2,Lateral Walks,1,3,1
3,Banded Goblet Squats,1,3,10
4,Goblet Lunges,1,3,10
...,...,...,...,...
8729,Mountain Climbers,1,3,30
8730,Back Squats,50,4,5
8731,Russian Twists,1,3,30
8732,RFE Lunges,1,3,8


In [68]:
exercises=big_df['exercise']
exercises=exercises.str.lower()
exercises=exercises.value_counts()
exercises

mountain climbers         468
burpees                   255
banded walks              250
j-jacks                   232
plank to pushup           177
                         ... 
trx triceps                 1
db oh triceps               1
bb deadlifts                1
banded weighted bridge      1
trx rows                    1
Name: exercise, Length: 527, dtype: int64

In [65]:
exercises[exercises>100].to_csv('top_exercises.csv')

In [83]:
exercises[exercises>100]

mountain climbers    468
burpees              255
banded walks         250
j-jacks              232
plank to pushup      177
plank circuit        174
side planks          163
db bench press       158
split squat jumps    145
dead bugs            139
sa db row            134
squat jumps          131
pushups              121
sl glute bridges     119
rfe lunges           111
bicycles             109
sa renegade row      101
Name: exercise, dtype: int64

In [84]:
# Simple Ratio
print(fuzz.ratio("sa renegade row", "db renegade row"))

# Partial Ratio
print(fuzz.partial_ratio("sa renegade row", "db renegade row"))


87
87


In [101]:
# Assuming unique_values is a list of your unique string values
unique_values = exercises.index.to_list()

# Threshold for similarity
similarity_threshold = 80

# Create a dictionary where the keys are strings and the values are lists of strings
similar_strings = defaultdict(list)

for i in range(len(unique_values)):
    for j in range(i + 1, len(unique_values)):
        similarity = fuzz.ratio(unique_values[i], unique_values[j])
        
        if similarity >= similarity_threshold:
            similar_strings[unique_values[i]].append(unique_values[j])

# Now similar_strings dictionary holds strings as keys and lists of similar strings as values


defaultdict(list,
            {'burpees': ['mc-burpees'],
             'plank to pushup': ['bosu plank to pushup'],
             'plank circuit': ['planks circuit',
              'plank circuiy',
              'p-plank circuit'],
             'side planks': ['side planks ', 'side plank'],
             'db bench press': ['db oh press',
              'bench press',
              'bb bench press',
              'sa db bench press',
              'sa bench press'],
             'split squat jumps': ['repeat squat jumps',
              'no flip split squat jumps',
              'spli squat jumps'],
             'dead bugs': ['yb dead bugs', 'ar dead bugs', 'deas bugs'],
             'sa db row': ['sa db row '],
             'squat jumps': ['squat jumps ', 'spli squat jumps'],
             'pushups': ['pushup'],
             'sl glute bridges': ['ffe sl glute bridges',
              'sl glute bridge',
              'glute bridges',
              'ffe sl glute bridge'],
             'rfe lung

### Data Sanitization
    1. The approach above is not perfect but it satisfies the general goal to consolidate the string exercises that are similar to each other. In most cases it does what I want, however, in some it combines it based on words that are not related to the exercise. See below
            'partner crunch holds': ['partner crunch hold'],
             'leg press machine': ['chest press machine',
              'oh press machine',
              'shoulder press machine'],

     Ultimately, as long as the top exercises are not being lost, that is what matters

In [110]:
# Get the counts of each string
counts = exercises.to_dict()

# Create a new dictionary to hold the counts for each string and its similar strings
combined_counts = {}

for key, similar_values in similar_strings.items():
    # Start with the count for the key string
    combined_counts[key] = counts[key]

    # Add the counts for each similar string
    for value in similar_values:
        if value in counts:
            combined_counts[key] += counts[value]
            # Once counted, remove it from the counts dictionary
            del counts[value]

# Now, add remaining items from counts dictionary to combined_counts
combined_counts.update(counts)


KeyError: 'db oh press'

In [107]:
combined_counts=pd.Series(combined_counts).sort_values(ascending=False)

In [108]:
combined_counts.loc['mountain climbers']

KeyError: 'mountain climbers'